In [1]:
import pandas as pd
from cuotasimss import CuotasImss
from imss import CalculoIMSSObrero, CalculoIMSSPatron
from isr import ISR
from datetime import datetime
from sbc import SBC
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_excel('./data/plantilla_laboral2.xlsx', sheet_name='Hoja1')
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso
0,1,Martha Rosa Alvarez Martínez,"33,256.68","1,500.00",2019-06-03
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50","1,297.00",2019-06-17
2,6,Karla Berenice Cano Garcia,"22,960.82","1,148.00",2019-07-23
3,7,Jonathan Covarrubias Sustaita,"21,719.58","1,086.00",2019-03-04
4,8,Miguel Dionicio Luna,"27,315.62","1,366.00",2018-01-09


In [3]:
df['Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%Y-%m-%d')

In [4]:
date_2022 = pd.to_datetime('2022/12/31')

In [5]:
df['SD'] = df['Sueldo mensual'].apply(lambda x: x / 30)
df['SBC'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo mensual'], vales = x['Vales'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['ISR'] = (df['Sueldo mensual'] * 1).apply(lambda x: ISR.function_isr(x))
df['IMSS_Ob'] = df['SBC'].apply(lambda x: CalculoIMSSObrero.function_imss_obrero(x))
df['IMSS_Pt'] = df.apply(lambda x: CalculoIMSSPatron.function_imss_patronal(x = x['SBC'], date=date_2022 ), axis=1)
df['ISN'] = df['Sueldo mensual'] * 0.03
_isr_ = df['Sueldo mensual'].apply(lambda x: ISR.function_isr(x))
df['Neto'] = df['Sueldo mensual'] - _isr_ - df['IMSS_Ob']
df['Aguinaldo'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['Prima Vacacional'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo mensual'], date_ingreso = x['Fecha Ingreso'], date_base = date_2022), axis=1)

In [6]:
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SD,SBC,ISR,IMSS_Ob,IMSS_Pt,ISN,Neto,Aguinaldo,Prima Vacacional
0,1,Martha Rosa Alvarez Martínez,"33,256.68","1,500.00",2019-06-03,"1,108.56","1,172.56","5,797.85",954.07,"6,308.16",997.70,"26,504.76","16,628.34","2,771.39"
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50","1,297.00",2019-06-17,864.38,910.00,"4,097.80",732.57,"5,007.64",777.94,"21,101.13","12,965.75","2,160.96"
2,6,Karla Berenice Cano Garcia,"22,960.82","1,148.00",2019-07-23,765.36,802.05,"3,463.27",641.51,"4,472.94",688.82,"18,856.04","11,480.41","1,913.40"
3,7,Jonathan Covarrubias Sustaita,"21,719.58","1,086.00",2019-03-04,723.99,758.70,"3,198.14",604.93,"4,258.19",651.59,"17,916.51","10,859.79","1,809.97"
4,8,Miguel Dionicio Luna,"27,315.62","1,366.00",2018-01-09,910.52,961.87,"4,400.52",776.33,"5,264.55",819.47,"22,138.77","13,657.81","2,731.56"


In [8]:
df.tail(4)

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SD,SBC,ISR,IMSS_Ob,IMSS_Pt,ISN,Neto,Aguinaldo,Prima Vacacional
49,65,José Alfredo López Madrigal,"17,687.64",884.00,2022-06-16,589.59,616.24,"2,336.92",484.76,"3,552.56",530.63,"14,865.96","4,775.66",0.00
50,66,Jesus Geovanni Ruvalcaba Mota,"17,687.64",884.00,2022-07-01,589.59,616.24,"2,336.92",484.76,"3,552.56",530.63,"14,865.96","4,421.91",0.00
51,67,Luis Manuel Guerrero Alcantara,"19,500.08",975.00,2022-08-22,650.00,679.38,"2,724.05",538.03,"3,865.33",585.00,"16,238.00","3,510.01",0.00
52,68,Shaila Barradas Santiago,"18,616.36",931.00,2022-09-19,620.55,648.60,"2,535.29",512.05,"3,712.83",558.49,"15,569.02","2,606.29",0.00


In [11]:
df.query('Nombre.str.contains("Ana Gómez")', engine='python')

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SD,SBC,ISR,IMSS_Ob,IMSS_Pt,ISN,Neto,Aguinaldo,Prima Vacacional
6,11,Ana Gómez Gallardo Aguilar,"60,500.00","1,500.00",2018-02-01,"2,016.67","2,126.98","13,369.45","1,759.22","11,035.63","1,815.00","45,371.33","30,250.00","6,050.00"
